In [ ]:
#use review data
x_r = range(2015,2021)
y1r =[100*sum(philly[philly.most_recent_review.dt.year==y].is_cbd)/len(philly[philly.most_recent_review.dt.year==y])
        for y in x_r]
y2r = [100*sum(philly[philly.earliest_review.dt.year==y].is_cbd)/len(philly[philly.earliest_review.dt.year==y])
        for y in x_r]
fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(121)
ax_o = fig.add_subplot(122)
ax.bar(x_r,[len(philly[philly.most_recent_review.dt.year==y]) for y in x_r],color="tab:cyan", width= 0.5 )
ax.bar(x_r,[sum(philly[philly.most_recent_review.dt.year==y].is_cbd) for y in x_r],color="tab:blue", width= 0.5 )
ax.set_xlabel("Closed",fontsize=14)
ax.set_ylabel("Business Count",fontsize=14)
#ax.set_ylim(0,1200)
ax2=ax.twinx()
ax2.plot(x_r,y1r, color="tab:red",marker="o" )
ax2.set_ylim(0,100)
ax.set_xticks(x_r, minor=False)
ax_o.bar(x_r,[len(philly[philly.earliest_review.dt.year==y]) for y in x_r],color="tab:cyan", width= 0.5 )
ax_o.bar(x_r,[sum(philly[philly.earliest_review.dt.year==y].is_cbd) for y in x_r],color="tab:blue", width= 0.5 )
ax_o.set_xlabel("Open",fontsize=14)
#ax_o.set_ylim(0,1200)
ax2=ax_o.twinx()
ax2.plot(x_r,y2r, color="tab:red",marker="o" )
ax2.set_ylabel("Percentage of CBD (%)",fontsize=14)
ax2.set_ylim(0,100)
ax_o.set_xticks(x_r, minor=False)

fig.suptitle(f'Change in Food Business Count ')
fig.supxlabel('Year')
plt.show()

In [ ]:
review_json_path = '../data/yelp_dataset/yelp_academic_dataset_review.json'
reviews = []
r_dtypes = {"stars": np.float16}
with open(review_json_path, "r") as f:
    reader = pd.read_json(f, orient="records", lines=True, 
                          dtype=r_dtypes, chunksize=1000)
        
    for chunk in tqdm(reader):
        reduced_chunk = chunk.drop(columns=['review_id', 'user_id'])\
                             .query("`date` >= '2015-12-01'")
        reviews.append(reduced_chunk)
    
reviews = pd.concat(reviews, ignore_index=True)

reviews_time = reviews.groupby(['business_id']).agg({'date': [np.min,np.max]}).droplevel(axis=1, level=0).reset_index()
reviews_time.rename({'amin':'earliest_review', 'amax':'most_recent_review'},axis=1,inplace=True)

In [ ]:
def define_status(row,status):
    l = []
    for y in years[1:]:
        if row[status+'_%s'%(y)] ==1:
            l.append()
            return str(y)
        else:
            continue
       
for s in ['close','open']:
    philly[s] = philly.apply(lambda x: define_status(x,s),axis=1)

In [ ]:
#use business license data
x = years[1:]
def all(status,y):
    return len(philly[philly[status]==str(y)])
def cbd(status,y):
    return sum(philly[philly[status]==str(y)].is_cbd)

y1_all = [all('close',y) for y in x]
y1_cbd = [cbd('close',y) for y in x]
y2_all = [all('open',y) for y in x]
y2_cbd = [cbd('open',y) for y in x]

y1 =[100*cbd('close',y)/all('close',y) for y in x]
y2 = [100* cbd('open',y)/all('open',y)  for y in x]

fig = plt.figure(figsize=(13,5))
ax = fig.add_subplot(121)
ax_o = fig.add_subplot(122)
ax.bar(x,y1_all,color="tab:cyan", width= 0.5 )
ax.bar(x,y1_cbd,color="tab:blue", width= 0.5 )

ax.set_xlabel("Closed",fontsize=14)
ax.set_ylabel("Business Count",fontsize=14)
ax.set_ylim(0,1800)
ax2=ax.twinx()
ax2.plot(x,y1, color="tab:red",marker="o" )
ax2.set_ylim(20,50)

ax.set_xticks(x, minor=False)
ax_o.bar(x,y2_all,color="tab:cyan", width= 0.5 )
ax_o.bar(x,y2_cbd,color="tab:blue", width= 0.5 )
ax_o.set_xlabel("New Open",fontsize=14)
ax_o.set_ylim(0,1800)
ax2=ax_o.twinx()
ax2.plot(x,y2, color="tab:red",marker="o" )
ax2.set_ylabel("Rate of Businesses in CBD (%)",fontsize=14)
ax2.set_ylim(20,50)
ax_o.set_xticks(x, minor=False)
ax.vlines(2019, 0, 4000, linestyles='--', color='b')
ax_o.vlines(2019, 0, 4000, linestyles='--', color='b')

fig.suptitle(f'Change in all Business Count')
fig.supxlabel('Year')
plt.savefig("../output/licenses.png")
plt.show()

In [ ]:
def gen_census(row):
    map = cg.coordinates(x=row.longitude , y=row.latitude)
    return map['2018 State Legislative Districts - Upper'][0]['GEOID']
philly['census_track']= philly.apply(gen_census, axis=1)

In [ ]:
cor = list(zip(philly.longitude,philly.latitude))
cen = []
for co in cor:
    info = cg.coordinates(x=co[0], y=co[1])
    code = info['2018 State Legislative Districts - Upper'][0]['GEOID']
    cen.append(code)

In [ ]:
df_tract['is_cbd'] = 0
d = {}
for c in df_tract.tract.unique():
    r = df_yelp[df_yelp.tract==c].is_cbd.sum()/len(df_yelp[df_yelp.tract==c].is_cbd)
    if r> 0.5:
        d[c] = 1
    else:
        d[c] = 0

df_tract['is_cbd'] = df_tract.apply(lambda x: d[x.tract], axis=1)
df_tract['post_covid'] = df_tract.apply(lambda x: 1 if x.year>=2020 else 0, axis=1)

In [ ]:
# fit model
x = df_tract.loc[:,['is_cbd', 'post_covid','DID']]
y = df_tract['New Open']
model = LinearRegression().fit(x, y)
X2 = sm.add_constant(x)
est = sm.OLS(y, X2).fit()
model= est.get_robustcov_results(cov_type='HC2', use_t=None)
print(model.summary())
#save output
plt.rc('figure', figsize=(12, 7))
plt.text(0.01, 0.05, str(model.summary()), {'fontsize': 10}, fontproperties = 'monospace') # approach improved by OP -> monospace!
plt.axis('off')
plt.tight_layout()
plt.savefig('../output/yelp_open.png')
plt.close()
write_path = '../output/yelp_open.csv'
with open(write_path, 'w') as f:
    f.write(model.summary().as_csv())

In [1]:
!pip install simplejson

You should consider upgrading via the 'C:\Users\dell\anaconda3\python.exe -m pip install --upgrade pip' command.


In [1]:
import simplejson as json
import numpy as np
import matplotlib.pyplot as plt

bx = file("nyc.json")
bx = json.loads(bx.read())

polygons = bx['features']


censusTracts = np.array([bx['features'][k]['geometry']['coordinates'][0] for k in range(len(bx['features']))]) 

for x in range(len(censusTracts)):
	plt.plot(np.array(censusTracts[x])[:, 0] , np.array(censusTracts[x])[:, 1]  , '-')

plt.xlim(xmin=900000, xmax=1100000)
plt.ylim(ymin=120000, ymax=280000)
plt.axes().set_aspect('equal', 'datalim')
plt.show()


NameError: name 'file' is not defined

In [ ]:
tracts = shapefile.Reader('../data/Philadelphia_Census_Tracts_2010/Philadelphia_Census_Tracts_2010.shp')
fields = [field[0] for field in tracts.fields[1:]]
attributes = []
geometry = []

for row in tracts.shapeRecords():
    geometry.append(shape(row.shape.__geo_interface__))
    attributes.append(dict(zip(fields, row.record)))
with open('../data/Philadelphia_Census_Tracts_2010/Philadelphia_Census_Tracts_2010.prj') as p:
    proj4 = osr.SpatialReference(p.read()).ExportToProj4()
print(proj4)
gdf = gpd.GeoDataFrame(data=attributes, geometry=geometry, crs=proj4)

In [ ]:
viz_df = data
for i in list(range(2013,2019))+[2020,2021]:
    name = 'INX'+str(i)
    viz_df[name] = 1*data.is_cbd
    viz_df.loc[viz_df['year'] != i, name] = 0
mod = lm.PanelOLS.from_formula('''new_open ~ 
INX2013 + INX2014 + 
 INX2015 + INX2016 + INX2017 + INX2018  + INX2020 + 
INX2021 +  
EntityEffects ''', viz_df)
clfe = mod.fit(cov_type = 'clustered',
cluster_entity = True)
res = pd.concat([clfe.params, clfe.std_errors], axis = 1)
res['ci'] = res['std_error']*1.96
res['year_index'] = list(range(2013,2019))+[2020,2021]
reference = pd.DataFrame([[0,0,0,2019]],
columns = ['parameter',
            'lower',
            'upper',
            'year_index'])
res = pd.concat([res, reference])
res = res.sort_values('year_index')
res['year'] = range(2013,2022)
plt.figure(figsize=(10, 5), facecolor='w')
plt.errorbar(x = 'year', y = 'parameter',
                    yerr = 'ci', data = res)
plt.axhline(0, linestyle = 'dashed')
plt.axvline(2019, linestyle = 'dashed', color='red')
plt.ylim(-20,20)
plt.xlabel('Year')
plt.ylabel('Diffreence-in-Diffrence Estimate (new open)')
plt.figtext(0.5, 0.01, "Source: https://www.opendataphilly.org/",position=(0.1,0.01))
plt.savefig('../output/did_o_trend.png')
plt.show()

In [ ]:
viz_df = data2
for i in list(range(2013,2019))+[2020,2021]:
    name = 'INX'+str(i)
    viz_df[name] = 1*data2.is_cbd
    viz_df.loc[viz_df['year'] != i, name] = 0
mod = lm.PanelOLS.from_formula('''close ~ INX2013 + INX2014 + 
 INX2015 + INX2016 + INX2017 + INX2018  + INX2020+ 
INX2021 +  
EntityEffects ''', viz_df)
clfe = mod.fit(cov_type = 'clustered',
cluster_entity = True)
res = pd.concat([clfe.params, clfe.std_errors], axis = 1)
res['ci'] = res['std_error']*1.96
res['year_index'] = list(range(2013,2019))+[2020,2021]
reference = pd.DataFrame([[0,0,0,2019]],
columns = ['parameter',
            'lower',
            'upper',
            'year_index'])
res = pd.concat([res, reference])
res = res.sort_values('year_index')
res['year'] = range(2013,2022)
plt.figure(figsize=(10, 5),facecolor='w')
plt.errorbar(x = 'year', y = 'parameter',
                    yerr = 'ci', data = res)
plt.axhline(0, linestyle = 'dashed')
plt.axvline(2019, linestyle = 'dashed', color='red')
plt.ylim(-15,15)
plt.xlabel('Year')
plt.figtext(0.5, 0.01, "Source: https://www.opendataphilly.org/",position=(0.1,0.01))
plt.ylabel('Diffreence-in-Diffrence Estimate (close)')
plt.savefig('../output/did_c_trend.png')
plt.show()

In [ ]:
viz_df = df_tract[df_tract.new_open_ln>0]
for i in list(range(2012,2019))+[2020,2021]:
    name = 'INX'+str(i)
    viz_df[name] = 1*df_tract[df_tract.new_open_ln>0].is_cbd
    viz_df.loc[viz_df['year'] != i, name] = 0

mod = lm.PanelOLS.from_formula('''new_open_ln ~  INX2012 + INX2013 + INX2014+
 INX2015 + INX2016 + INX2017 + INX2018  + INX2020 + 
INX2021 + 
EntityEffects ''', viz_df)
clfe = mod.fit(cov_type = 'clustered',
cluster_entity = True)
# Get coefficients and CIs
res = pd.concat([clfe.params, clfe.std_errors], axis = 1)
res['ci'] = res['std_error']*1.96
# Add our quarter values
res['year_index'] = list(range(2012,2019))+[2020,2021]
reference = pd.DataFrame([[0,0,0,2019]],
columns = ['parameter',
            'lower',
            'upper',
            'year_index'])
res = pd.concat([res, reference])
res = res.sort_values('year_index')
res['year'] = range(2012,2022)

plt.figure(figsize=(10, 5),facecolor='w')
plt.errorbar(x = 'year', y = 'parameter',
                    yerr = 'ci', data = res)
plt.axhline(0, linestyle = 'dashed')
plt.axvline(2019, linestyle = 'dashed', color='red')
plt.ylim(-0.5,0.5)
plt.xlabel('Year',ha='right')
plt.ylabel('Diffreence-in-Diffrence Estimate (ln(new open))')
plt.figtext(0.5, 0.01, "Source: https://www.yelp.com/dataset",position=(0.1,0.01))
plt.savefig('../output/did_o_trend_ln_y.png')
plt.show()

In [ ]:
viz_df = df_tract
for i in list(range(2012,2019))+[2020,2021]:
    name = 'INX'+str(i)
    viz_df[name] = 1*df_tract.is_cbd
    viz_df.loc[viz_df['year'] != i, name] = 0

mod = lm.PanelOLS.from_formula('''new_open ~  INX2012 + INX2013 + INX2014+
 INX2015 + INX2016 + INX2017 + INX2018  + INX2020 + 
INX2021 + 
EntityEffects ''', viz_df)
clfe = mod.fit(cov_type = 'clustered',
cluster_entity = True)
res = pd.concat([clfe.params, clfe.std_errors], axis = 1)
res['ci'] = res['std_error']*1.96
res['year_index'] = list(range(2012,2019))+[2020,2021]
reference = pd.DataFrame([[0,0,0,2019]],
columns = ['parameter',
            'lower',
            'upper',
            'year_index'])
res = pd.concat([res, reference])
res = res.sort_values('year_index')
res['year'] = range(2012,2022)
plt.figure(figsize=(10, 5),facecolor='w')
plt.errorbar(x = 'year', y = 'parameter',
                    yerr = 'ci', data = res)
plt.axhline(0, linestyle = 'dashed')
plt.axvline(2019, linestyle = 'dashed', color='red')
plt.ylim(-4,3)
plt.xlabel('Year',ha='right')
plt.ylabel('Diffreence-in-Diffrence Estimate (new open)')
plt.figtext(0.5, 0.01, "Source: https://www.yelp.com/dataset",position=(0.1,0.01))
plt.savefig('../output/did_o_trend_y.png')
plt.show()

In [ ]:
viz_df = df_tract
for i in list(range(2015,2019))+[2020,2021]:
    name = 'INX'+str(i)
    viz_df[name] = 1*df_tract.is_cbd
    viz_df.loc[viz_df['year'] != i, name] = 0

mod = lm.PanelOLS.from_formula('''close ~ 
 INX2015 + INX2016 + INX2017 + INX2018  + INX2020 + 
INX2021 + 
EntityEffects ''', viz_df)
clfe = mod.fit(cov_type = 'clustered',
cluster_entity = True)
res = pd.concat([clfe.params, clfe.std_errors], axis = 1)
res['ci'] = res['std_error']*1.96
res['year_index'] = list(range(2015,2019))+[2020,2021]
reference = pd.DataFrame([[0,0,0,2019]],
columns = ['parameter',
            'lower',
            'upper',
            'year_index'])
res = pd.concat([res, reference])
res = res.sort_values('year_index')
res['year'] = range(2015,2022)
plt.figure(figsize=(10, 5),facecolor='w')
plt.errorbar(x = 'year', y = 'parameter',
                    yerr = 'ci', data = res)
plt.axhline(0, linestyle = 'dashed')
plt.axvline(2019, linestyle = 'dashed', color='red')
plt.ylim(-4,8)
plt.xlabel('Year',ha='right')
plt.ylabel('Diffreence-in-Diffrence Estimate (close)')
plt.figtext(0.5, 0.01, "Source: https://www.yelp.com/dataset",position=(0.1,0.01))
plt.savefig('../output/did_c_trend_y.png')
plt.show()

In [ ]:
#(1)all businesses
fig = plt.figure(figsize=(13,5),facecolor='w')
ax_o = fig.add_subplot(121)
ax_o.plot(years[6:],[count('open', y,philly)[1] for y in years[6:]],color="tab:cyan" )
ax_o.plot(years[6:],[count('open', y,philly)[3] for y in years[6:]],color="tab:blue" )
ax_o.set_title("New open",fontsize=14)
ax_o.set_ylim(0,2500)
ax_o.set_ylabel("Business Count",fontsize=14)
ax2=ax_o.twinx()
ax2.plot(years[6:],[count('open', y,philly)[2] for y in years[6:]], color="tab:red",marker="o" )
ax2.set_ylim(20,50)
ax_o.set_xticks(years[1:], minor=False)
ax = fig.add_subplot(122)
ax.plot(years[6:],[count('close', y,philly)[1] for y in years[6:]],color="tab:cyan")
ax.plot(years[6:],[count('close', y,philly)[3] for y in years[6:]],color="tab:blue")
ax.set_title("Close-down",fontsize=14)
ax.set_ylim(0,2500)
ax2=ax.twinx()
ax2.plot(years[6:],[count('close', y,philly)[2] for y in years[6:]], color="tab:red",marker="o" )
ax2.set_ylim(20,50)
ax2.set_ylabel("Rate of Businesses in CBD (%)",fontsize=14)
ax.set_xticks(years[6:], minor=False)
ax.vlines(2019, 0, 2500, linestyles='--', color='b')
ax_o.vlines(2019, 0, 2500, linestyles='--', color='b')
fig.supxlabel('Year')
plt.figtext(0.5, 0.01, "Source: https://www.yelp.com/dataset",position=(0.1,-0.01))
plt.savefig("../output/yelp.png")
plt.show()

In [ ]:
#(2) Food Businesses
food= philly_geo[philly_geo.categories.notna()][philly_geo[philly_geo.categories.notna()].categories.str.contains('food|restaurant|bar|cafe|Grocery', case=False)]
fig = plt.figure(figsize=(13,5),facecolor='w')
ax_o = fig.add_subplot(121)
ax_o.plot(years[6:],[count('open', y,food)[1] for y in years[6:]],color="tab:cyan")
ax_o.plot(years[6:],[count('open', y,food)[3] for y in years[6:]],color="tab:blue")
ax_o.set_title("New open",fontsize=14)
ax_o.set_ylim(0,1200)
ax_o.set_ylabel("Business Count",fontsize=14)
ax2=ax_o.twinx()
ax2.plot(years[6:],[count('open', y,food)[2] for y in years[6:]], color="tab:red",marker="o" )
ax2.set_ylim(20,50)
ax_o.set_xticks(years[6:], minor=False)
ax = fig.add_subplot(122)
ax.plot(years[6:],[count('close', y,food)[1] for y in years[6:]],color="tab:cyan")
ax.plot(years[6:],[count('close', y,food)[3] for y in years[6:]],color="tab:blue")
ax.set_title("Close-down",fontsize=14)
ax.set_ylim(0,1200)
ax2=ax.twinx()
ax2.plot(years[6:],[count('close', y,food)[2] for y in years[6:]], color="tab:red",marker="o" )
ax2.set_ylim(20,50)
ax2.set_ylabel("Rate of Businesses in CBD (%)",fontsize=14)
ax.set_xticks(years[6:], minor=False)
ax.vlines(2019, 0, 1200, linestyles='--', color='b')
ax_o.vlines(2019, 0, 1200, linestyles='--', color='b')
fig.supxlabel('Year')
plt.figtext(0.5, 0.01, "Source: https://www.yelp.com/dataset",position=(0.1,-0.01))
plt.savefig("../output/yelp_food.png")
plt.show()

In [ ]:
#use business license data
x = [i for i in range(2017,2022)]
y1_all = [len(food_license[food_license.close==str(y)]) for y in x]
y1_cbd = [sum(food_license[food_license.close==str(y)].is_cbd) for y in x]
y1_noncbd = [(len(food_license[food_license.close==str(y)])- sum(food_license[food_license.close==str(y)].is_cbd)) for y in x]
# y1_all.pop(6)
# y1_all.insert(6, (y1_all[5]+y1_all[7])/2)
# y1_noncbd.pop(6)
# y1_noncbd.insert(6, (y1_noncbd[5]+y1_noncbd[7])/2)
# y1_cbd.pop(6)
# y1_cbd.insert(6, (y1_cbd[5]+y1_cbd[7])/2)
y1 =[100*sum(food_license[food_license.close==str(y)].is_cbd)/len(food_license[food_license.close==str(y)])
        for y in x]
y2 = [100*sum(food_license[food_license.open==str(y)].is_cbd)/len(food_license[food_license.open==str(y)])
        for y in x]
fig = plt.figure(figsize=(13,5),facecolor='w')

ax_o = fig.add_subplot(121)
#ax_o.bar(x,[len(food_license[food_license.open==str(y)]) for y in x],color="tab:cyan", width= 0.5 )
ax_o.plot(x,[sum(food_license[food_license.open==str(y)].is_cbd) for y in x],color="tab:blue")
ax_o.plot(x,[(len(food_license[food_license.open==str(y)])- sum(food_license[food_license.open==str(y)].is_cbd)) for y in x],color="tab:blue")
ax_o.set_title("New open",fontsize=14)
ax_o.set_ylabel("Business Count",fontsize=14)
ax_o.set_ylim(0,2000)
ax2=ax_o.twinx()
ax2.plot(x,y2, color="tab:red",marker="o" )
ax2.set_ylim(0,50)
ax_o.set_xticks(x[1::2], minor=False)
ax_o.vlines(2019, 0, 2000, linestyles='--', color='b')
ax = fig.add_subplot(122)
#ax.bar(x,y1_all,color="tab:cyan", width= 0.5 )
ax.plot(x,y1_cbd,color="tab:blue")
ax.plot(x,y1_noncbd,color="tab:blue")
ax.set_title("Close-down",fontsize=14)
ax.set_ylim(0,2000)
ax2=ax.twinx()
ax2.plot(x,y1, color="tab:red",marker="o" )
ax2.set_ylim(0,50)
ax2.set_ylabel("Rate of Businesses in CBD (%)",fontsize=14)
ax.set_xticks(x[1::2], minor=False)
ax.vlines(2019.5, 0, 2000, linestyles='--', color='b')
fig.supxlabel('Year')
plt.figtext(0.5, 0.01, "Source: https://www.opendataphilly.org/",position=(0.1,-0.01))
plt.savefig("../output/licenses.png")
plt.show()

In [ ]:
df_missing = df_new[df_new.tract.isna()]
df_missing.reset_index(inplace=True,drop=True)
df_missing = df_missing[['latitude','longitude']] 
df_missing = df_missing.drop_duplicates()
df_missing= df_missing[df_missing.latitude.notna()]
df_missing.reset_index(inplace=True,drop=True)
df_missing.to_csv('../output/df_missing.csv', index=False) 

In [ ]:
df_use = food_license[['censustrac' , 'open', 'close', 'is_cbd']]
df_cen = pd.DataFrame(df_use.censustrac.value_counts())
df_cen.reset_index(inplace=True)
df_cen.rename({"index":"census_track", "censustrac": "count"}, inplace=True, axis=1)
cen_trac = df_use.censustrac.unique()
df_use['close'][df_use.close=='3013'] = '2013'

In [ ]:
c_lst = []
for c in cen_trac:
    df = pd.DataFrame(df_use[df_use.censustrac==c].open.value_counts())
    df.reset_index(inplace=True)
    df.rename({"index":"year","open":"open_count"}, inplace=True, axis=1)
    df['census_track'] = c
    df['is_cbd'] = df_use[df_use.censustrac==c].is_cbd.max()
    
    c_lst.append(df)
data = pd.concat(c_lst, ignore_index=True)
data['post_covid'] = data.apply(lambda x: 1 if int(x.year)>=2020 else 0, axis=1)
#creating interaction variable
data['DID'] = data.apply( lambda x: 1 if (x['is_cbd']==1) and (x['post_covid']==1) else 0, axis=1)
# create log open count
data['open_ln'] = np.log(data.open_count)
census_dic = dict(data.census_track.value_counts())

c_lst = []
for c in cen_trac:
    df = pd.DataFrame(df_use[df_use.censustrac==c].close.value_counts())
    df.reset_index(inplace=True)
    df.rename({"index":"year","close":"close_count"}, inplace=True, axis=1)
    df['census_track'] = c
    df['is_cbd'] = df_use[df_use.censustrac==c].is_cbd.max()
    
    c_lst.append(df)
data2 = pd.concat(c_lst, ignore_index=True)
data2['post_covid'] = data2.apply(lambda x: 1 if int(x.year)>=2020 else 0, axis=1)
#creating interaction variable
data2['DID'] = data2.apply( lambda x: 1 if (x['is_cbd']==1) and (x['post_covid']==1) else 0, axis=1)
# create log open count
data2['close_ln'] = np.log(data2.close_count)
census_dic2 = dict(data2.census_track.value_counts())
data['census'] = data.apply(lambda x: str(int(float(x.census_track) * 100)), axis=1)
data2['census'] = data2.apply(lambda x: str(int(float(x.census_track) * 100)), axis=1)
data.rename({'open_count' :'New open'}, inplace=True,axis='columns')
data2.rename({'close_count' :'Close-down'}, inplace=True,axis='columns')
data['census_trac']= data.census_track.astype(float)
data2['census_trac']= data2.census_track.astype(float)
data['year']= data.year.astype(int)
data2['year']= data2.year.astype(int)
data = data.merge(pop,left_on = ['census_trac','year'], right_on=['tr','yr'])
data2 = data2.merge(pop, left_on = ['census_trac','year'], right_on=['tr','yr'])
data = data.merge(area,left_on = ['census_trac'], right_on=['tr'])
data2 = data2.merge(area, left_on = ['census_trac'], right_on=['tr'])
data['pop'] = data.population/1000
data2['pop'] = data2.population/1000
data['density'] = data.population*1000/data.SHAPE_AREA
data2['density'] = data2.population*1000/data.SHAPE_AREA
data = data[data.year!=2012]
data2 = data2[data2.year!=2012]
data['post covid * density'] = data['density']  * data['post_covid'] 
data2['post covid * density'] = data2['density']  * data2['post_covid'] 

In [ ]:
data.to_csv('../output/bl_ct.csv', index=False)
data2.to_csv('../output/bl_ct2.csv', index=False)

In [ ]:
#use business license data
x = [i for i in range(2006,2022)]
y1_all = [len(food_license[food_license.close==str(y)]) for y in x]
y1_cbd = [sum(food_license[food_license.close==str(y)].is_cbd) for y in x]
y1_all.pop(6)
y1_all.insert(6, (y1_all[5]+y1_all[7])/2)
y1_cbd.pop(6)
y1_cbd.insert(6, (y1_cbd[5]+y1_cbd[7])/2)
y1 =[100*sum(food_license[food_license.close==str(y)].is_cbd)/len(food_license[food_license.close==str(y)])
        for y in x]
y2 = [100*sum(food_license[food_license.open==str(y)].is_cbd)/len(food_license[food_license.open==str(y)])
        for y in x]
fig = plt.figure(figsize=(13,5),facecolor='w')

ax_o = fig.add_subplot(121)
ax_o.bar(x,[len(food_license[food_license.open==str(y)]) for y in x],color="tab:cyan", width= 0.5 )
ax_o.bar(x,[sum(food_license[food_license.open==str(y)].is_cbd) for y in x],color="tab:blue", width= 0.5 )
ax_o.set_title("New open",fontsize=14)
ax_o.set_ylabel("Business Count",fontsize=14)
ax_o.set_ylim(0,2000)
ax2=ax_o.twinx()
ax2.plot(x,y2, color="tab:red",marker="o" )
ax2.set_ylim(0,50)
ax_o.set_xticks(x[1::2], minor=False)
ax_o.vlines(2019, 0, 2000, linestyles='--', color='b')
ax = fig.add_subplot(122)
ax.bar(x,y1_all,color="tab:cyan", width= 0.5 )
ax.bar(x,y1_cbd,color="tab:blue", width= 0.5 )
ax.set_title("Close-down",fontsize=14)
ax.set_ylim(0,2000)
ax2=ax.twinx()
ax2.plot(x,y1, color="tab:red",marker="o" )
ax2.set_ylim(0,50)
ax2.set_ylabel("Rate of Businesses in CBD (%)",fontsize=14)
ax.set_xticks(x[1::2], minor=False)
ax.vlines(2019, 0, 2000, linestyles='--', color='b')
fig.supxlabel('Year')
plt.figtext(0.5, 0.01, "Source: https://www.opendataphilly.org/",position=(0.1,-0.01))
plt.savefig("../output/licenses.png")
plt.show()

In [ ]:
# fit model with log(open_count)
x = df_tract3.loc[:,['is_cbd', 'post_covid','DID','density']][df_tract3['ln(New open)']>=0]
y_ln = df_tract3['ln(New open)'][df_tract3['ln(New open)']>=0]
X2 = sm.add_constant(x)
est_l_o = sm.OLS(y_ln, X2).fit(cov_type='HC2', use_t=None)

In [ ]:
# fit model with log(close_count)
x = df_tract3.loc[:,['is_cbd', 'post_covid','DID','density']][df_tract3['ln(Close-down)']>=0]
y_ln = df_tract3['ln(Close-down)'][df_tract3['ln(Close-down)']>=0]
X2 = sm.add_constant(x)
est_l_c = sm.OLS(y_ln, X2).fit(cov_type='HC2', use_t=None)